# Import

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
from time import time

from tensorflow.keras.datasets import cifar10

# DenseNet 121

In [2]:
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model

In [3]:
input_shape = 32, 32, 3
n_classes = 10

In [4]:
model = densenet(input_shape,n_classes)
model.compile( loss=tf.keras.losses.sparse_categorical_crossentropy ,optimizer=tf.keras.optimizers.Adam( lr=0.0001 ) ,metrics=[ 'acc' ])
model.summary()
print(len(model.layers))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 8, 8, 64)     0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 8, 8, 64)     256         max_pooling2d[0][0]              
______________________________________________________________________________________________

# Data

In [5]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

# normalize to range 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

val_x = train_x[:5000]
val_y = train_y[:5000]

# Train model

In [6]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', verbose=1)
list_cb = [es]

In [7]:

start = time()
model.fit( train_x , train_y , epochs=50, batch_size=1024, validation_split=0.3, callbacks=list_cb)
training_time = time()-start

Epoch 1/50
35/35 [==============================] - 2515s 69s/step - loss: 2.0582 - acc: 0.2513 - val_loss: 2.3161 - val_acc: 0.1014
Epoch 2/50
35/35 [==============================] - 2365s 68s/step - loss: 1.4973 - acc: 0.4576 - val_loss: 2.4090 - val_acc: 0.1014
Epoch 3/50
35/35 [==============================] - 1352s 38s/step - loss: 1.2786 - acc: 0.5490 - val_loss: 2.5794 - val_acc: 0.1014
Epoch 4/50
35/35 [==============================] - 1418s 40s/step - loss: 1.0846 - acc: 0.6300 - val_loss: 2.7232 - val_acc: 0.1014
Epoch 5/50
35/35 [==============================] - 1407s 40s/step - loss: 0.8755 - acc: 0.7116 - val_loss: 2.7715 - val_acc: 0.1017
Epoch 6/50
35/35 [==============================] - 1417s 40s/step - loss: 0.6669 - acc: 0.7948 - val_loss: 2.9266 - val_acc: 0.1033
Epoch 7/50
35/35 [==============================] - 1372s 39s/step - loss: 0.4470 - acc: 0.8757 - val_loss: 2.8473 - val_acc: 0.1207
Epoch 8/50
35/35 [==============================] - 1337s 38s/step - 

# Evaluate

In [8]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 89s 282ms/step - loss: 3.1953 - acc: 0.2490


[3.195298671722412, 0.24899999797344208]

In [9]:
model.evaluate(train_x, train_y)

1563/1563 [==============================] - 347s 222ms/step - loss: 3.1539 - acc: 0.2554


[3.1538822650909424, 0.2553600072860718]

In [10]:
training_time

17296.929596185684